In [ ]:
import numpy as np
from data_with_notebook.scripts.util_mnist_reader import load_mnist
import tensorflow as tf
from functools import partial
import matplotlib.pyplot as plt

PATH = "data_with_notebook/data/fashion"     # Data Path

In [ ]:
from ANN import *

'''
Artifitial Neural Network

This is a Neural Network with 1 hidden layer
Inputs are 60000 images with 784 pixels
Output has 10 categories
The prediction is based on the percentage of predictions
The highest percentages among all the prediction is the predicted answer.
'''
### Hyper Parameters used to adjust the result ###
hiddens = [[20,],[80,],[300,]]
learning_rate = [0.001, 0.0001, 0.00001]
activation = 'sigmoid'
learning_rate_descent = 1
epoch = 10
batch_size = 6000

### Data ###
X,y = get_data(PATH, kind='train', norm_size=255)
val_X,val_y = get_validation_data(X,y,0.2)      ### use 20% of the training data for validation ###
test_X,test_y = get_data(PATH, kind='t10k', norm_size=255)

### Neural Network Model ###
ann = ANN(X.shape[1],hiddens[1],10,activation)
ann.__initialize_weights_biases__()     
ann.__initialize_hyperparameters__(
  learning_rate[1],
  learning_rate_descent,
  epoch,
  batch_size)
ann.__train__(X,y)

plt.plot(np.arange(int(len(X) / batch_size)*epoch),ann.accuracy_)

ann.__test__(test_X,test_y)

In [ ]:
from MLANN import *

'''
Multilayers Neural Network with Keras
Cross Entropy is used for the loss function,
Stochasitc Gradient Descent is used for the optimizer
Accuracy is used to measure the model accuracy
'''     
### Hyper Parameters used to adjust the result ###
hiddens = [[500,100],[300,50],[500,300,100]]
batch_size = [600,5000,10000]

activation = 'sigmoid'
epoch = 100

### Data ###
X,y = get_data(PATH, kind='train', norm_size=255)
val_X,val_y = get_validation_data(X,y,0.2)      ### use 20% of the training data for validation ###
test_X,test_y = get_data(PATH, kind='t10k', norm_size=255)


### Neural Network Model with Keras ###
mlann = MLANN(X.shape[1],hiddens[0],10,activation)
mlann.__build_keras_ann__()
mlann.__train__(X,y,(val_X,val_y),epoch,batch_size[1])

plt.plot(np.arange(len(mlann.training_info_.history['acc'])),mlann.training_info_.history['acc'])

mlann.__test__(test_X,test_y)

In [ ]:
from CNN import *

def get_validation_data(X,y,validation_percentage):
    validation_X = []
    validation_y = []
    random_index = np.random.permutation(len(X))
    val_size = int(len(X) * validation_percentage)
    for i in range(val_size):
        validation_X.append(X[random_index[i]])
        validation_y.append(y[random_index[i]])
    return np.array(validation_X), np.array(validation_y)

'''
This function initiazes the default layer settings for each type of layers
So everytime a new layer is created there will not be repetitive coding
'''
def get_partial():
    def_conv2d = partial(Conv2d_Parameters,filters_=10,kernel_size_=3,strides_=(1,1),padding_='SAME',activation_='relu')
    def_pool = partial(Pooling_Parameters,pooling_type_='max', pooling_size_=2)
    def_dense = partial(Dense_Parameters,units_=64, activation_='relu')
    return def_conv2d, def_pool, def_dense

'''
Parameters for the CNN model
These parameters can be adjusted for a better model
'''
def get_cnn_params():
    def_conv2d, def_pool, def_dense = get_partial()
    cnn_input = def_conv2d()
    cnn_convs = [def_conv2d()]
    cnn_pools = [def_pool()]
    cnn_hids = [def_dense()]
    cnn_output = def_dense(units_=10,activation_='softmax')
    return cnn_input, cnn_convs, cnn_pools, cnn_hids, cnn_output




'''
Convolutional Neural Network with Keras
This class creates a convolutional Neural Network with that consists:
input layaer, convolutional layers, pooling layers, falltened Neural Network Layers, and output layer
'''
### Hyper Parameters used to adjust the result ###
### More Hyper parameters can be adjusted from 'get_partial' function on line 171 ###
epoch = 1000
batch_size = 1000
### The parameter classes for CNN class ###
cnn_input, cnn_convs, cnn_pools, cnn_hids, cnn_output = get_cnn_params()


### Data ###
X,y = get_data(PATH, kind='train', norm_size=255)
np.random.shuffle(X);np.random.shuffle(y)
val_X,val_y = get_validation_data(X,y,0.2)      ### use 20% of the training data for validation ###
X = np.reshape(X,(X.shape[0],28,28,1))          ### Reshape the flattened image to 3D ###
val_X = np.reshape(val_X,(val_X.shape[0],28,28,1))
test_X,test_y = get_data(PATH, kind='t10k', norm_size=255)
test_X = np.reshape(test_X,(test_X.shape[0],28,28,1))


### Convolutional Neural Network ###
cnn = CNN(cnn_input, cnn_convs, cnn_pools, cnn_hids, cnn_output)
cnn.__build_cnn_model__()
cnn.__train__(X,y,(val_X,val_y),epoch,batch_size)

plt.plot(np.arange(len(cnn.training_info_.history['acc'])),cnn.training_info_.history['acc'])

cnn.__test__(test_X,test_y)